## Investigations Concerning Covariance Matrix Computations and Optimal Kernel Hyperparameters

In [1]:
using Pkg; Pkg.add("GaussianProcesses")


   Resolving package versions...
   Installed GR_jll ─────────── v0.72.9+1
   Installed XZ_jll ─────────── v5.4.4+0
   Installed CodeTracking ───── v1.3.4
   Installed JLLWrappers ────── v1.5.0
   Installed Compat ─────────── v4.9.0
   Installed Unitful ────────── v1.16.3
   Installed SpecialFunctions ─ v2.3.1
   Installed DataStructures ─── v0.18.15
   Installed JuliaSyntax ────── v0.4.6
   Installed JuliaInterpreter ─ v0.9.24
   Installed OpenSSL_jll ────── v1.1.22+0
   Installed InverseFunctions ─ v0.1.12
   Installed URIs ───────────── v1.5.0
   Installed ColorSchemes ───── v3.23.0
    Updating `~/.julia/environments/v1.9/Project.toml`
  [891a1506] + GaussianProcesses v0.12.5
    Updating `~/.julia/environments/v1.9/Manifest.toml`
  [49dc2e85] - Calculus v0.5.1
  [da1fd8a2] ↑ CodeTracking v1.3.1 ⇒ v1.3.4
  [35d6a980] ↑ ColorSchemes v3.22.0 ⇒ v3.23.0
  [34da2185] ↑ Compat v4.8.0 ⇒ v4.9.0
  [864edb3b] ↑ DataStructures v0.18.14 ⇒ v0.18.15
  [b4f34e82] + Distances v0.10.9
  [fa6b7ba4] 

In [2]:
using GaussianProcesses
using Random

Random.seed!(20140430)
# Training data
n = 10;                          #number of training points
x = 2π * rand(n);              #predictors
y = sin.(x) + 0.05*randn(n);   #regressors

[ Info: Precompiling GaussianProcesses [891a1506-143c-57d2-908e-e1f8e92e6de9]


In [7]:
#Select mean and covariance function
mZero = MeanZero()                   #Zero mean function
kern = SE(0.0,0.0)                   #Sqaured exponential kernel (note that hyperparameters are on the log scale)

logObsNoise = -1.0                        # log standard deviation of observation noise (this is optional)
gp = GP(x,y,mZero,kern,logObsNoise) 
gp = GP(x,y,mZero,kern) 

GP Exact object:
  Dim = 1
  Number of observations = 10
  Mean function:
    Type: MeanZero, Params: Float64[]
  Kernel:
    Type: SEIso{Float64}, Params: [0.0, 0.0]
  Input observations = 
[2.3008941978115995 4.177929157381414 … 2.015277043100713 4.730977535989725]
  Output observations = [0.7710763105152831, -0.8391332919835048, -0.10379941216625867, 0.8742717770868422, -0.5504286710256119, -0.6641636221208442, -0.3429297010103842, 0.6259817552095643, 0.9198567208087357, -0.883006046800485]
  Variance of observation noise = 0.01831563888873418
  Marginal Log-Likelihood = -2.624

In [8]:
gp.cK

10×10 PDMats.PDMat{Float64, Matrix{Float64}}:
 1.01832      0.171765    0.000611588  …  0.960032     0.0522011
 0.171765     1.01832     0.143668        0.0964691    0.858189
 0.000611588  0.143668    1.01832         0.000195688  0.366509
 0.981966     0.117898    0.000288281     0.995513     0.0322426
 0.00278378   0.299235    0.91692         0.00100323   0.606317
 0.320927     0.934061    0.0648248    …  0.200298     0.653496
 0.502339     0.780732    0.0280494       0.344927     0.454037
 0.281109     0.00242728  3.74719e-7      0.425376     0.000305647
 0.960032     0.0964691   0.000195688     1.01832      0.0250341
 0.0522011    0.858189    0.366509        0.0250341    1.01832

In [9]:
using Optim
optimize!(gp; method=ConjugateGradient())

 * Status: success

 * Candidate solution
    Final objective value:     -5.314356e+00

 * Found with
    Algorithm:     Conjugate Gradient

 * Convergence measures
    |x - x'|               = 3.63e-08 ≰ 0.0e+00
    |x - x'|/|x'|          = 1.76e-09 ≰ 0.0e+00
    |f(x) - f(x')|         = 8.03e-12 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 1.51e-12 ≰ 0.0e+00
    |g(x)|                 = 1.87e-10 ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    64
    f(x) calls:    164
    ∇f(x) calls:   113


In [10]:
gp.cK

10×10 PDMats.PDMat{Float64, Matrix{Float64}}:
 0.387643     0.0539713    9.81307e-5  …  0.370345     0.0142314
 0.0539713    0.387643     0.0441917      0.0282976    0.326661
 9.81307e-5   0.0441917    0.387643       2.74103e-5   0.126052
 0.379828     0.0354205    4.22886e-5     0.385697     0.00829952
 0.000535109  0.100457     0.351782       0.000170755  0.221425
 0.108642     0.35915      0.0181352   …  0.0641006    0.240797
 0.179385     0.293846     0.00710122     0.117775     0.160194
 0.0936713    0.000459021  2.4892e-8      0.14892      4.51498e-5
 0.370345     0.0282976    2.74103e-5     0.387643     0.00625251
 0.0142314    0.326661     0.126052       0.00625251   0.387643